<a href="https://colab.research.google.com/github/promptmule4real/promptmule_demo/blob/main/PromptMule_demo_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Improved PromptMule API Demo Suite v1.2

Enhanced with better error handling, user-friendly input mechanisms, and expanded functionality. This tool offers a more comprehensive exploration of PromptMule's API capabilities.

For more information, visit [PromptMule Documentation](https://promptmule.com/docs).
"""

import requests
import json
import pandas as pd
import random

# Constants for API interactions
BASE_URL = 'https://api.promptmule.com/'
ENDPOINTS = {
    'PROMPT': 'prompt',
    'LIST_API_KEYS': 'api-keys/',
    'USAGE': 'usage',
    'DAILY_STATS': 'usage/daily-stats'
}

# Enhanced display options for Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Function Definitions

def safe_request(callable, *args, **kwargs):
    """Wrapper function for API calls to handle exceptions."""
    try:
        response = callable(*args, **kwargs)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"API request failed: {e}")
        return None

def login_to_promptmule(api_key, username, password):
    """Improved login function with better error handling."""
    endpoint = BASE_URL + "login"
    headers = {"Content-Type": "application/json", "x-api-key": api_key}
    body = {"username": username, "password": password}
    response = safe_request(requests.post, endpoint, headers=headers, json=body)
    return response.get("token") if response else ""

def generate_prompt(user_message, variation):
    """Generates a prompt message based on user input and variation number."""
    base_prompt = "As a developer using OpenAI, share a tip for cache-as-a-service implementation: "
    return f"{base_prompt}{user_message}. Variation: {variation}"

def track_semantic_scores(response_json):
    """Extracts semantic scores and associated data from the API's response."""
    return [(choice['prompt-id'], choice.get('score'), choice['message']['content'], choice['index']) for choice in response_json['choices']]

def send_prompt_requests(api_key, user_message, semantic_similarity, max_response_quantity, variations, dataframe):
    """Sends prompt requests to the API and updates the dataframe with responses."""
    headers = {"Content-Type": "application/json", "x-api-key": api_key}
    for variation in range(1, variations + 1):
        prompt_message = generate_prompt(user_message, variation)
        body = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": prompt_message}],
            "max_tokens": "50",
            "temperature": "0.2",
            "api": "openai",
            "semantic": str(semantic_similarity),
            "sem_num": str(max_response_quantity)
        }
        response = safe_request(requests.post, BASE_URL + ENDPOINTS['PROMPT'], headers=headers, json=body)
        if response:
            choices_data = track_semantic_scores(response)
            for choice in choices_data:
                dataframe.loc[len(dataframe)] = [choice[0], semantic_similarity, choice[1], choice[2], choice[3]]

# Main Script Execution

def main():
    api_key = input("Enter your PromptMule API Key: ")
    user_message = input("Enter your development focus (e.g., 'improving user experience'): ")

    print("authenticating using demo account...")
    token = login_to_promptmule(api_key, "demo", "Abcd@1234!")
    if not token:
        print("Authentication failed. Exiting script.")
        return

    df = pd.DataFrame(columns=['Prompt ID', 'Similarity Target', 'Similarity Score', 'Response', 'Response Index'])
    semantic_nums = [1.0, 0.75, 0.5]
    max_responses = random.choice(range(1, 11))
    variations = 3

    print("Starting prompt creation process...")

    for sem_num in semantic_nums:
        send_prompt_requests(api_key, user_message, sem_num, max_responses, variations, df)

    print("\nPrompt-ing Report:")
    print(df.to_string(index=True))

#if __name__ == "__main__":
main()

# Additional API calls and features can be incorporated following similar patterns

# End of Script
